# 파이썬 코딩의 기술 - ch4. 컴프리헨션과 제너레이터

## Better way 27. map과 filter 대신 컴프리헨션을 사용하라

In [2]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9]

# wrong 1
squares = []
for x in a:
    squares.append(x**2)
print(squares)

print('============')
# wrong 2
squares = list(map(lambda x: x**2, a))
print(squares)

print('============')
# right
squares = [x**2 for x in a]
print(squares)

[1, 4, 9, 16, 25, 36, 49, 64, 81]
[1, 4, 9, 16, 25, 36, 49, 64, 81]
[1, 4, 9, 16, 25, 36, 49, 64, 81]


In [8]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9]

# wrong
squares_even = list(map(lambda x: x**2, filter(lambda x: x % 2 == 0, a)))
print(squares_even)

print('============')
# right
squares_even = [x**2 for x in a if x % 2 == 0]
print(squares_even)

[4, 16, 36, 64]
[4, 16, 36, 64]


## 27-2. 리스트, 딕셔너리, 집합, 이터레이션 컴프리헨션

In [9]:
list_comprehension = [x**2 for x in range(10)]
dict_comprehension = {x: x**2 for x in range(10)}
set_comprehension = {x**2 for x in range(10)}
iter_comprehension = (x**2 for x in range(10))

print(list_comprehension)
print(dict_comprehension)
print(set_comprehension)
print(iter_comprehension)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
{0: 0, 1: 1, 2: 4, 3: 9, 4: 16, 5: 25, 6: 36, 7: 49, 8: 64, 9: 81}
{0, 1, 64, 4, 36, 9, 16, 49, 81, 25}
<generator object <genexpr> at 0x7f1a7a0a8d60>


## Better way 28. 컴프리헨션 내부에 제어 하위 식을 세 개 이상 사용하지 말라

제어 하위 식 : x <u>for x in range(10)</u>

In [12]:
tensor = [[[1, 2, 3], [4, 5, 6]], 
          [[7, 8, 9], [10, 11, 12]]]

# wrong
flat = [x for matrix in tensor for rows in matrix for x in rows]
print(flat)

print('============')
# right
flat = []
for matrix in tensor:
    for rows in matrix:
        for x in rows:
            flat.append(x)
print(flat)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


## Better way 29. 대입식을 사용해 컴프리헨션 안에서 반복 작업을 피하라

In [14]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9]

# wrong
modulus_even = [x % 3 for x in a if (x % 3) % 2 == 0]
print(modulus_even)

print('============')
# right
modulus_even = [modulus for x in a if (modulus := x % 3) % 2 == 0]
print(modulus_even)

[2, 0, 2, 0, 2, 0]
[2, 0, 2, 0, 2, 0]


## Better way 30. 리스트를 변환하기보다는 제너레이터를 사용하라

In [3]:
string = '1101011011011011'

# wrong
def find_zero_index1(string):
    result = []
    for index, char in enumerate(string):
        if char == '0':
            result.append(index)
    return result

print(find_zero_index1(string))

print('============')
# right
def find_zero_index2(string):
    for index, char in enumerate(string):
        if char == '0':
            yield index

for index in find_zero_index2(string):
    print(index, end=' ')

[2, 4, 7, 10, 13]
2 4 7 10 13 

## Better way 31. 인자에 대해 이터레이션 할 때는 방어적이 돼라

함수의 인자로 이터레이션이 들어오면 한 번밖에 사용하지 못한다.

이터레이터 프로토콜(iterator protocol)을 구현한 새로운 컨테이너 클래스를 인자로 넘기자.

In [21]:
def divide_even_odd(numbers):
    for number in numbers:
        if number % 2 == 0:
            yield number
    for number in numbers:
        if number % 2 != 0:
            yield number

# wrong
print('divide_even_odd list :', end=' ')
for number in divide_even_odd([i for i in range(10)]):
    print(number, end=' ')
    
print('\ndivide_even_odd iter :', end=' ')
for number in divide_even_odd((i for i in range(10))):
    print(number, end=' ')

print('\n============')
# right
class NumberContainer:
    def __init__(self, numbers):
        self.numbers = [number for number in numbers]

    def __iter__(self):
        for number in self.numbers:
            yield number

print('divide_even_odd NumberContainer(list) :', end=' ')
for number in divide_even_odd(NumberContainer([i for i in range(10)])):
    print(number, end=' ')
    
print('\ndivide_even_odd NumberContainer(iter) :', end=' ')
for number in divide_even_odd(NumberContainer((i for i in range(10)))):
    print(number, end=' ')

divide_even_odd list : 0 2 4 6 8 1 3 5 7 9 
divide_even_odd iter : 0 2 4 6 8 
divide_even_odd NumberContainer(list) : 0 2 4 6 8 1 3 5 7 9 
divide_even_odd NumberContainer(iter) : 0 2 4 6 8 1 3 5 7 9 

collections.abc의 Iterator 클래스를 이용하여

함수 내에서 반복 사용 가능한 컨테이너인지 체크하는 방법도 좋다.

In [18]:
from collections.abc import Iterator

def safe_divide_even_odd(numbers):
    if isinstance(numbers, Iterator):
        raise TypeError('컨테이너를 제공해야 합니다.')
    for number in numbers:
        if number % 2 == 0:
            yield number
    for number in numbers:
        if number % 2 != 0:
            yield number

print('safe_divide_even_odd list :', end=' ')
for number in safe_divide_even_odd([i for i in range(10)]):
    print(number, end=' ')
    
print('\nsafe_divide_even_odd iter :', end=' ')
for number in safe_divide_even_odd((i for i in range(10))):
    print(number, end=' ')

safe_divide_even_odd list : 0 2 4 6 8 1 3 5 7 9 
safe_divide_even_odd iter : 

TypeError: ignored

## Better way 32. 긴 리스트 컴프리헨션보다는 제너레이터 식을 사용하라

In [19]:
# wrong
[i**2 for i in range(100)]

# right
(i**2 for i in range(100))

<generator object <genexpr> at 0x7faa0879ca50>

## Better way 33. yield from을 사용해 여러 제너레이터를 합성하라

yield from (제너레이터) : 제너레이터에서 하나씩 가져와 배출

In [26]:
# wrong
def even_odd_iter1(numbers):
    for number in numbers:
        if number % 2 == 0:
            yield number
    for number in numbers:
        if number % 2 != 0:
            yield number

for number in even_odd_iter1(range(10)):
    print(number, end=' ')

print('\n============')
# right
def even_iter(numbers):
    for number in numbers:
        if number % 2 == 0:
            yield number

def odd_iter(numbers):
    for number in numbers:
        if number % 2 != 0:
            yield number

def even_odd_iter2(numbers):
    yield from even_iter(numbers)
    yield from odd_iter(numbers)

for number in even_odd_iter2(range(10)):
    print(number, end=' ')

0 2 4 6 8 1 3 5 7 9 
0 2 4 6 8 1 3 5 7 9 

## Better way 34. send로 제너레이터에 데이터를 주입하지 말라

합성할 제너레이터들의 입력으로 이터레이터를 전달하는 방식이 send를 사용하는 방식보다 더 낫다.

send는 가급적 사용하지 말라.

## Better way 35. 제너레이터 안에서 throw로 상태를 변화시키지 말라

제너레이터에서 throw를 사용해 예외를 발생시키기보다는 __iter__ 메서드를 구현하는 클래스를 사용해 예외적인 경우에 상태를 전이시키는 것이 낫다.

## Better way 36. 이터레이터나 제너레이터를 다룰 때는 itertools를 사용하라

- 여러 이터레이터 연결하기
  - chain : 연결
  - repeat : n회 반복
  - cycle : 계속 반복 : 무한 루프 주의
  - tee : 이터레이터 n개 복사
  - zip_longest : 두 이터레이터 중 긴 값에 맞춤
- 이터레이터에서 원소 거르기
  - islice : 이터레이터 슬라이싱
  - takewhile : while
  - dropwhile : takewhile의 반대
  - filterfalse : filter의 반대
- 이터레이터에서 원소의 조합 만들어내기
  - accumulate : 합성 (default : +)
  - product : 데카르트 곱
  - permutations : 순열
  - combinations : 조합
  - combinations_with_replacement : 중복 조합

In [51]:
import itertools

# chain : 연결
it = itertools.chain([1, 2, 3], [4, 5, 6])
print(f'chain([1, 2, 3], [4, 5, 6]) : {list(it)}')

# repeat : n회 반복
it = itertools.repeat([1, 2, 3], 3)
print(f'repeat([1, 2, 3], 3) : {list(it)}')

# cycle : 계속 반복
it = itertools.cycle([1, 2, 3])
result = [next(it) for _ in range(10)]
print(f'cycle([1, 2, 3]) : {result}')

# tee : 이터레이터 n개 복사
it1, it2, it3 = itertools.tee([1, 2, 3], 3)
print(f'it1 : {list(it1)}')
print(f'it2 : {list(it2)}')
print(f'it3 : {list(it3)}')

# zip_longest : 두 이터레이터 중 긴 값에 맞춤
it1 = [1, 2, 3]
it2 = [5, 6, 7, 8, 9]
it3 = itertools.zip_longest(it1, it2, fillvalue='?')
print(f'zip_longest(it1, it2, fillvalue=\'?\') : {list(it3)}')

# islice : 이터레이터 슬라이싱
it = [1, 2, 3, 4, 5, 6, 7, 8, 9]
it_sliced = itertools.islice(it, 1, 7, 2)
print(f'islice(it, 1, 7, 2) : {list(it_sliced)}')

# takewhile : do-while
it = [1, 2, 3, 4, 5, 6, 7, 8, 9]
it_takewhile = itertools.takewhile(lambda x: x < 5, it)
print(f'takewhile(lambda x: x < 5, it) : {list(it_takewhile)}')

# dropwhile : takewhile의 반대
it = [1, 2, 3, 4, 5, 6, 7, 8, 9]
it_dropwhile = itertools.dropwhile(lambda x: x < 5, it)
print(f'dropwhile(lambda x: x < 5, it) : {list(it_dropwhile)}')

# filterfalse : filter의 반대
it = [1, 2, 3, 4, 5, 6, 7, 8, 9]
it_filterfalse = itertools.filterfalse(lambda x: x % 2 == 0, it)
print(f'filterfalse(it, lambda x: x % 2) : {list(it_filterfalse)}')

# accumulate : 합성 (default : +)
it = [1, 2, 3, 4, 5, 6, 7, 8, 9]
it_accumulate1 = itertools.accumulate(it)
print(f'accumulate(it) : {list(it_accumulate1)}')
it_accumulate2 = itertools.accumulate(it, lambda a, b: a - b)
print(f'accumulate(it, lambda a, b: a - b) : {list(it_accumulate2)}')

# product : 데카르트 곱
it1 = [1, 2, 3]
it2 = [4, 5, 6]
it_product = itertools.product(it1, it2)
print(f'product(it1, it2) : {list(it_product)}')

# permutations : 순열
it = [1, 2, 3]
it_permutations = itertools.permutations(it)
print(f'permutations(it) : {list(it_permutations)}')

# combinations : 조합
it = [1, 2, 3]
it_combinations = itertools.combinations(it, 2)
print(f'combinations(it, 2) : {list(it_combinations)}')

# combinations_with_replacement : 중복 조합
it = [1, 2, 3]
it_combinations_with_replacement = itertools.combinations_with_replacement(it, 2)
print(f'combinations_with_replacement(it, 2) : {list(it_combinations_with_replacement)}')

chain([1, 2, 3], [4, 5, 6]) : [1, 2, 3, 4, 5, 6]
repeat([1, 2, 3], 3) : [[1, 2, 3], [1, 2, 3], [1, 2, 3]]
cycle([1, 2, 3]) : [1, 2, 3, 1, 2, 3, 1, 2, 3, 1]
it1 : [1, 2, 3]
it2 : [1, 2, 3]
it3 : [1, 2, 3]
zip_longest(it1, it2, fillvalue='?') : [(1, 5), (2, 6), (3, 7), ('?', 8), ('?', 9)]
islice(it, 1, 7, 2) : [2, 4, 6]
takewhile(lambda x: x < 5, it) : [1, 2, 3, 4]
dropwhile(lambda x: x < 5, it) : [5, 6, 7, 8, 9]
filterfalse(it, lambda x: x % 2) : [1, 3, 5, 7, 9]
accumulate(it) : [1, 3, 6, 10, 15, 21, 28, 36, 45]
accumulate(it, lambda a, b: a - b) : [1, -1, -4, -8, -13, -19, -26, -34, -43]
product(it1, it2) : [(1, 4), (1, 5), (1, 6), (2, 4), (2, 5), (2, 6), (3, 4), (3, 5), (3, 6)]
permutations(it) : [(1, 2, 3), (1, 3, 2), (2, 1, 3), (2, 3, 1), (3, 1, 2), (3, 2, 1)]
combinations(it, 2) : [(1, 2), (1, 3), (2, 3)]
combinations_with_replacement(it, 2) : [(1, 1), (1, 2), (1, 3), (2, 2), (2, 3), (3, 3)]
